In [ ]:
import tensorflow as tf
import numpy as np

# Constants
g = 9.8
m_earth = 5.972e24
k = 6.67430e-11  # Gravitational constant

# Data generation
def generate_data(samples=1000):
    n = np.random.randint(2, 11, size=samples)
    r = np.random.uniform(1, 100, size=samples)
    m_object = np.random.uniform(1, 1000, size=samples)
    m_n = 2 * m_earth
    g_n = n * g
    F_n = k * (m_object * m_n) / (r ** (n - 1))
    return n, r, m_object, g_n, F_n

n, r, m_object, g_n, F_n = generate_data()

# Normalize data
inputs = np.vstack((n, r, m_object, g_n)).T
outputs = F_n
inputs = (inputs - inputs.mean(axis=0)) / inputs.std(axis=0)
outputs = (outputs - outputs.mean()) / outputs.std()

# Define PINN model
class PINN(tf.keras.Model):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden = [
            tf.keras.layers.Dense(64, activation='tanh'),
            tf.keras.layers.Dense(64, activation='tanh'),
            tf.keras.layers.Dense(64, activation='tanh'),
        ]
        self.output_layer = tf.keras.layers.Dense(1, activation='linear')

    def call(self, inputs):
        x = inputs
        for layer in self.hidden:
            x = layer(x)
        return self.output_layer(x)

# Loss functions
def physics_loss(model, n, r, m_object, g_n):
    m_n = 2 * m_earth
    F_pred = model(tf.stack([n, r, m_object, g_n], axis=1))
    physics = F_pred - k * (m_object * m_n) / (r ** (n - 1))
    return tf.reduce_mean(tf.square(physics))

def combined_loss(model, inputs, outputs):
    n, r, m_object, g_n = inputs[:, 0], inputs[:, 1], inputs[:, 2], inputs[:, 3]
    data_loss = tf.reduce_mean(tf.square(model(inputs) - outputs))
    phys_loss = physics_loss(model, n, r, m_object, g_n)
    return data_loss + 0.1 * phys_loss  # Lambda = 0.1

# Train the model
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(inputs, outputs):
    with tf.GradientTape() as tape:
        loss = combined_loss(model, inputs, outputs)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training loop
inputs_tensor = tf.convert_to_tensor(inputs, dtype=tf.float32)
outputs_tensor = tf.convert_to_tensor(outputs, dtype=tf.float32)

for epoch in range(1000):
    loss = train_step(inputs_tensor, outputs_tensor)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}")


In [ ]:
# Predict
predictions = model(inputs_tensor).numpy()

# Plot results
import matplotlib.pyplot as plt
plt.scatter(outputs, predictions, alpha=0.5)
plt.xlabel("Actual F_n")
plt.ylabel("Predicted F_n")
plt.title("PINN Results")
plt.show()
